## Import

In [12]:
# Install packages
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install git+https://github.com/openai/whisper.git 
!pip install pydub
!pip install wave
!pip install numpy
!pip install pandas

Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torchvision-0.15.2%2Bcu118-cp38-cp38-win_amd64.whl (4.9 MB)
  Using cached https://download.pytorch.org/whl/cu118/torchaudio-2.0.2%2Bcu118-cp38-cp38-win_amd64.whl (2.5 MB)
  Using cached https://download.pytorch.org/whl/Pillow-9.3.0-cp38-cp38-win_amd64.whl (2.5 MB)


In [1]:
import wave
import numpy as np
import pandas as pd
import whisper
import os

from pydub import AudioSegment

## Data Preprocessing

In [5]:
# ../audio/ 폴더에 최초의 영상에서 추출한 음성을 저장
# 해당 폴더에 존재하는 음성 파일들의 파일명을 모두 불러와 리스트 형태로 저장

folder_path = '../Data/audio/'
file_list = os.listdir(folder_path)

In [6]:
# whisper 모델은 m4a 형식이 아닌 wav 형식을 사용
# 이에 맞춰 기존의 m4a 형식을 wav 형식으로 변환
# 변환 후 ../wav_audio/ 폴더에 변환된 음성 파일들을 저장

for file in file_list:
    # m4a 파일 로드
    audio = AudioSegment.from_file("../Data/audio/" + file, format="m4a")

    # wav 파일로 저장
    audio.export("../Data/wav_audio/" + file.split('.')[0] + '.wav', format="wav")

In [2]:
# 변환된 wav 파일들을 ../wav_audio/ 폴더에서 불러옴
# 해당 폴더에 존재하는 음성 파일들의 파일명을 모두 불러와 리스트 형태로 저장

folder_path = '../Data/wav_audio/'
file_list = os.listdir(folder_path)

## Create Word vocabulary

In [3]:
# 대한씨름협회 유튜브에 존재하는 '55가지 씨름 공식 기술 소개│씨름 기술 55수' 영상에 존재하는 씨름의 기술들을 사용하여 씨름 기술 단어 사전을 생성
# 단어들을 technique이라는 변수에 리스트로 저장

technique = '''
앞무릎치기
앞무릎짚기
모둠앞무릎치기
앞무릎뒤집기
앞무릎짚고밀기
뒷무릎치기
옆무릎치기
뒷무릎
앞무릎
옆무릎
오른옆무릎치기
콩꺾기
오금당기기
앞다리들기
손짚이기
호미걸이
낚시걸이
뒤축걸이
뒷발목걸이
앞다리차기
무릎대어돌리기
연장걸이
빗장걸이
밭다리걸기
밭다리치기
밭다리감아돌리기
안다리걸기
왼안다리걸기
덧걸이
덮걸이
오금걸이
안다리
밭다리
배지기
왼배지기
오른배지기
엉덩배지기
차돌리기
어깨걸어치기
업어던지기
들배지기
들베지기
들어튕겨배지기
들며차내어배지기
들며튕겨배지기
들어잡채기
잡치기
후려던지기
들어찧기
들안아놓기
들어낚시걸이
들어호미걸이
뿌려치기
밀어치기
등채기
등쳐감아돌리기
애목잡치기
정면뒤집기
뒤집기
측면뒤집기
목감아뒤집기
끌어치기
꼭뒤집기
앞으로누르기
통안아넘기기
허리꺾기
땡기기
'''
tech_list = list(technique.split('\n'))
tech_list = tech_list[1:-1]

## Predict

In [4]:
# whisper 모델을 불러옴
# whisper 모델이 사용하는 checkpoint 중 가장 용량이 큰 'large' checkpoint를 불러와서 사용
# model이라는 변수에 해당 모델을 저장

model = whisper.load_model("large")

In [5]:
# tech_df라는 데이터프래임 생성
# 해당 변수에는 파일명과 기술 사용 횟수가 저장될 예정

tech_df = pd.DataFrame(columns=['title','tech_count'])

# result_list라는 빈 리스트 생성
# 해당 리스트에는 model이 STT를 수행한 결과인 Text를 리스트 형태로 저장

result_list = []

# decode_options라는 dictionary 생성
# whisper는 다양한 언어를 지원하기 때문에 언어를 지정하지 않으면 한국어가 아닌 다른 나라의 언어로 예측할 위험이 있음
# 이를 방지하기 위해 language를 한국어로 고정하는 변수를 생성

decode_options = {'language' : 'ko'}

In [6]:
for idx, file_ in enumerate(file_list): # file_list에 저장된 wav 파일들을 하나씩 불러온다.
    file = folder_path + file_ # file_list에는 파일명만 존재하기 때문에 파일을 정상적으로 불러오기 위해 파일 경로를 더해줌으로써 파일을 정상적으로 불러온다.
    title = list(file.split(' '))[1] # 파일명은 '(Audio) 금강_16_1_1_0.wav'와 같은 형태로 구성되어 있기 때문에 띄어쓰기를 기준으로 분할 후 2번째 단어를 저장한다.
    title = list(title.split('.'))[0] # 위에서 저장한 단어에서 .을 기준으로 다시 분할한 후 중요한 부분인 1번째 단어를 최종 title로 저장


    # whisper 모델에 불러온 음성 파일을 적용
    # initial_prompt를 통해 우리가 주로 탐색하고자 하는 단어들을 미리 지정
    # **decode_options을 통해 언어를 한국어로 고정
    result = model.transcribe(file, initial_prompt='들배지기 안다리 밭다리 잡치기 오금당기기 배지기 맞배지기 밀어치기 호미걸이 빗장걸이 덫걸이 차돌리기', 
                              **decode_options)
    
    # model의 output은 음성에서 나오는 모든 문장을 하나의 str 형태로 반환하기 때문에 이를 띄어쓰기 기준으로 분할하여 단어별로 리스트 형태로 저장
    words = list(result['text'].split(' '))

    # model의 output을 result_list에 저장
    result_list.append(result['text'])

    # 음성에서 나온 씨름의 기술을 저장하기 위한 빈 리스트 생성
    appear_list = []

    for word in words: # 단어 리스트를 하나씩 불러온다.
        # 띄어쓰기를 기준으로 구분하여 아무 것도 존재하지 않는 부분이 생긴다.
        # 이를 건너띄게 한다.
        if word == '':
            continue

        # 씨름 기술 단어를 정상적으로 추출하기 위해 단어 뒤에 주로 붙는 것들을 제거
        if word[-1] in ['.', '!', ',','을','를','?','는','은']:
            word = word[:-1]

        # 단어가 씨름 기술 단어 사전에 존재하거나 단어가 다음과 같은 리스트에 존재하는 것으로 끝난다면 모두 기술로 판단하고 리스트에 저장
        if word in tech_list or word[-2:] in ['지기','치기','걸이','걸기','리기','다리', '거리']:
            appear_list.append(word)

    # 만약 기술이 하나도 없다면 0을 반환
    if appear_list == []:
        tech_df.loc[idx] = [title, 0]
        continue

    # 중복으로 여러 번 나온 기술을 1개로 합치기 위한 알고리즘
    i = 0
    while i != (len(appear_list) - 1):
        if appear_list[i] == appear_list[i+1]:
            del appear_list[i]
        else:
            i += 1

    # 데이터프레임에 title과 기술 사용 횟수를 저장
    tech_df.loc[idx] = [title, len(appear_list)]

## Evaluation

In [16]:
# 직접 라벨링을 통해 기술 횟수를 기록한 csv 파일을 불러옴
score_df = pd.read_csv('../Data/score.csv', encoding='cp949')

# model이 생성한 데이터프레임과 위에서 불러온 csv 파일을 merge하여 합침
predict_df = pd.merge(score_df, tech_df)

# 정확도를 계산하는 함수 생성
def get_score(df):
    return min(df['tech_count'], (df['score'])) / max(df['tech_count'], (df['score']))

# 정확도 계산
accuracy = predict_df.apply(get_score, axis=1)

print('Accruacy :', np.mean(accuracy))

Accruacy : 0.8750526810050621


In [ ]:
# whisper.csv에 최종적으로 출력된 데이터프레임을 저장
tech_df.to_csv('../Output/whisper.csv', index=False)